<h1><center> 1. Projet-7 Nettoyage & Analyse Exploratoire</center></h1>

In [1]:
import mlflow
import gc
from hyperopt import hp
from hyperopt import tpe
from hyperopt import fmin
from hyperopt import space_eval, STATUS_OK

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

import lightgbm as lgb
import lime
from lime import lime_tabular

import os

import matplotlib as mpl
%matplotlib inline
import matplotlib.pyplot as plt
import missingno as msno
import numpy as np
import pandas as pd
import pickle 
import random
import re
import seaborn as sns
import shap

import sklearn
from sklearn.dummy import DummyClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression

from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

import time

# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')



Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


In [2]:
# # numpy and pandas for data manipulation
# import numpy as np
# import pandas as pd 

# # sklearn preprocessing for dealing with categorical variables
# from sklearn.preprocessing import LabelEncoder

# # File system manangement
# import os

# # Suppress warnings 
# import warnings
# warnings.filterwarnings('ignore')

# # matplotlib and seaborn for plotting
# import matplotlib.pyplot as plt
# import seaborn as sns

In [3]:
dico_pd_option = {
    'display.max_rows': 400,
    'display.max_column': 200,
    'display.width': 300,
    'display.precision': 4,
    'display.max_colwidth': 1000,
    'mode.chained_assignment': None
}
for cle, val in dico_pd_option.items():
    pd.set_option(cle, val)

## Les fonctions

In [4]:
def get_missing_values(df_work, pourcentage, affiche_heatmap):
    # 1. Nombre de valeurs manquantes totales
    nb_nan_tot = df_work.isna().sum().sum()
    nb_donnees_tot = np.product(df_work.shape)
    pourc_nan_tot = round((nb_nan_tot / nb_donnees_tot) * 100, 2)
    print(
        f'Valeurs manquantes :{nb_nan_tot} NaN pour\
        {nb_donnees_tot} données ({pourc_nan_tot} %)')
    if pourcentage:
        print("-------------------------------------------------------------")
        print("Nombre et pourcentage de valeurs manquantes par variable\n")
        # 2. Visualisation du nombre et du pourcentage de valeurs manquantes
        # par variable
        values = df_work.isnull().sum()
        percentage = 100 * values / len(df_work)
        table = pd.concat([values, percentage.round(2)], axis=1)
        table.columns = [
            'Nombres de valeurs manquantes',
            '% de valeurs manquantes']
        display(table[table['Nombres de valeurs manquantes'] != 0]
                .sort_values('% de valeurs manquantes', ascending=False)
                .style.background_gradient('seismic'))
    if affiche_heatmap:
        print("-------------------------------------------------------------")
        print("Heatmap de visualisation des valeurs manquantes")
        # 3. Heatmap de visualisation des valeurs manquantes
        plt.figure(figsize=(20, 10))
        sns.heatmap(df_work.isna(), cbar=False)
        plt.show()

In [5]:
def description_variables(dataframe, type_var='all'):
    df_taille = dataframe.shape[0]

    if type_var == 'num':
        desc_var = dataframe.describe(include=[np.number])
    elif type_var == 'cat':
        desc_var = dataframe.describe(
            include='object')
    elif type_var == 'date':
        desc_var = dataframe.describe(
            include='datetime64[ns]')
    elif type_var == 'bool':
        desc_var = dataframe.describe(
            include='bool')
    else:
        desc_var = dataframe.describe(include='all')

    desc_type = pd.DataFrame(
        dataframe[desc_var.columns].dtypes, columns=['type']).T
    nb_nan = df_taille - desc_var.loc['count'].T
    pourcentage_nan = nb_nan * 100 / df_taille
    desc_nan = pd.DataFrame([nb_nan, pourcentage_nan],
                            index=['nb_nan', '%_nan'])
    desc_var = pd.concat([desc_type, desc_nan, desc_var])

    return desc_var

In [6]:
def cleaning_categories(col):
    '''
    Fonction qui nettoie une phrase de ses caractères spéciaux.
    Et la passe en majuscule( pour un passage en nom de colonne avec one hot)
    '''
    col = str(col).replace('-', '').replace('+', '').replace('/', ' ').replace(':', '_')
    col = str(col).upper()
    return col

## Importation data

In [7]:
# List files available
print(os.listdir("data/cleaned"))

['test.csv', 'train.csv']


In [8]:
train = pd.read_csv('data/cleaned/train.csv')
test = pd.read_csv('data/cleaned/test.csv')
print('Training data shape: ', train.shape)
print('Training data shape: ', test.shape)

Training data shape:  (246005, 122)
Training data shape:  (61502, 122)


In [9]:
train.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,364258,0,REVOLVING LOANS,F,N,Y,0,90000.0,202500.0,10125.0,202500.0,UNACCOMPANIED,WORKING,HIGHER EDUCATION,MARRIED,HOUSE APARTMENT,0.0106,-16115,-7774,-7787.0,-4385,NaN,1,1,0,1,1,0,CORE STAFF,2.0,2,2,THURSDAY,10,0,0,0,0,0,0,SCHOOL,NaN,0.6141,0.7571,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NAN,NAN,NaN,NAN,NAN,0.0,0.0,0.0,0.0,-1426.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,416252,0,CASH LOANS,M,Y,N,0,225000.0,625536.0,38398.5,540000.0,UNACCOMPANIED,WORKING,SECONDARY SECONDARY SPECIAL,MARRIED,HOUSE APARTMENT,0.0097,-10538,-592,-9590.0,-2557,11.0,1,1,0,1,1,0,DRIVERS,2.0,2,2,WEDNESDAY,14,0,0,0,0,0,0,SELFEMPLOYED,0.1931,0.2177,0.1385,0.2598,0.2017,0.9811,0.7416,0.0389,0.28,0.2414,0.3333,0.375,0.2307,0.2118,0.2985,0.0,0.0,0.2647,0.2093,0.9811,0.7517,0.0393,0.282,0.2414,0.3333,0.375,0.2359,0.2314,0.311,0.0,0.0,0.2623,0.2017,0.9811,0.7451,0.0392,0.28,0.2414,0.3333,0.375,0.2347,0.2155,0.3038,0.0,0.0,REG OPER ACCOUNT,BLOCK OF FLATS,0.2938,PANEL,NO,0.0,0.0,0.0,0.0,-688.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,4.0
2,190280,0,CASH LOANS,M,Y,Y,1,180000.0,247675.5,22846.5,229500.0,UNACCOMPANIED,WORKING,SECONDARY SECONDARY SPECIAL,MARRIED,HOUSE APARTMENT,0.0031,-9386,-2184,-4217.0,-2025,21.0,1,1,1,1,1,0,LABORERS,3.0,3,3,SUNDAY,10,0,0,0,0,1,1,BUSINESS ENTITY TYPE 3,NaN,0.5146,0.5674,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NAN,NAN,NaN,NAN,NAN,1.0,0.0,1.0,0.0,-587.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,102484,0,CASH LOANS,M,N,Y,0,225000.0,969579.0,54139.5,837000

## Feature engineering

In [17]:
def clean_data(train, test):

    # Gerer les valeurs abérantes
    train['DAYS_EMPLOYED_ANOM'] = train["DAYS_EMPLOYED"] == 365243
    test['DAYS_EMPLOYED_ANOM'] = test["DAYS_EMPLOYED"] == 365243
    train['DAYS_EMPLOYED'].replace({365243: np.nan}, inplace = True)
    test["DAYS_EMPLOYED"].replace({365243: np.nan}, inplace = True)
    
    # Extraire les identifiants
    train_id = train['SK_ID_CURR']
    test_id = test['SK_ID_CURR']
    
    # Extraire les targets
    labels = train['TARGET']
    
    # Supprimer les identifiants de train(et target) et test 
    train = train.drop(columns = ['SK_ID_CURR', 'TARGET'])
    test = test.drop(columns = ['SK_ID_CURR'])

    # Extraire le nom des colonnes
    columns_name = list(train.columns)
    
    # Imputation des Nan
    imputer = SimpleImputer(strategy = 'median')
    train = imputer.fit_transform(train.select_dtypes(np.number))
    test = imputer.transform(test.select_dtypes(np.number))
    
#     # Ecodage des categories
#     # Create a label encoder object
#     le = LabelEncoder()
#     le_count = 0

#     # Iterate through the columns
#     for col in train:
#         if train[col].dtype == 'object':
#             # If 2 or fewer unique categories
#             if len(list(train[col].unique())) <= 2:
#                 # Train on the training data
#                 le.fit(train[col])
#                 # Transform both training and testing data
#                 train[col] = le.transform(train[col])
                
#                 # Keep track of how many columns were label encoded
#                 le_count += 1
                
#     print('%d columns were label encoded.' % le_count)
    
#     # one-hot encoding of categorical variables
#     train = pd.get_dummies(train)

#     print('Training Features shape: ', train.shape)   
    
    
#     # test 
#     le_count = 0
#     for col in test:
#         if test[col].dtype == 'object':
#             # If 2 or fewer unique categories
#             if len(list(test[col].unique())) <= 2:
#                 # Train on the training data
#                 le.fit(test[col])
#                 # Transform both training and testing data
#                 test[col] = le.transform(test[col])
                
#                 # Keep track of how many columns were label encoded
#                 le_count += 1
                
#     print('%d columns were label encoded.' % le_count)
    
#     # one-hot encoding of categorical variables
#     test = pd.get_dummies(test)

#     print('Training Features shape: ', test.shape)  
    
#     # Création des features Moyennes, %...

#     # StandardScaler 
#     cols_to_scale = train.columns.drop('TARGET')
#     train[cols_to_scale] = StandardScaler().fit_transform(train[cols_to_scale])
#     test[cols_to_scale] = StandardScaler().fit_transform(test[cols_to_scale])
    
    # retourner 2 dataset propre train et test
    return train, test

In [ ]:
imputer = SimpleImputer(strategy = 'median')
train = imputer.fit_transform(train.select_dtypes(np.number))

In [19]:
train_clean, test_clean = clean_data(train, test)

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- TARGET


# Modélisation

## Baseline

In [ ]:
# Grid Search

In [ ]:
from sklearn.dummy import DummyClassifier

clf = DummyClassifier(strategy="most_frequent")
clf.fit(X_train_sample, y_train_sample)
y_pred = clf.predict(X_test)

In [ ]:
def plot_roc_curve(classifiers, X, y, n_splits=5):

    kf = KFold(n_splits=n_splits)    
    for name_clf, clf in classifiers.items():
        print(name_clf)
        tprs = []
        aucs = []
        mean_fpr = np.linspace(0, 1, 100)
        for i, (train, test) in enumerate(kf.split(X, y)):

            clf.fit(X.iloc[train, :], y.iloc[train])

            y_pred_grd = clf.predict_proba(X.iloc[test, :])[:, 1]
            fpr, tpr, thresholds = roc_curve(y.iloc[test], y_pred_grd)

            interp_tpr = np.interp(mean_fpr, fpr, tpr)
            interp_tpr[0] = 0.0
            tprs.append(interp_tpr)
            thresholds = auc(fpr, tpr)

            aucs.append(thresholds)

        mean_tpr = np.mean(tprs, axis=0)
        mean_tpr[-1] = 1.0
        mean_auc = auc(mean_fpr, mean_tpr)
        std_auc = np.std(aucs)
        plt.plot(mean_fpr, mean_tpr,
            label=name_clf + r'(AUC = %0.2f $\pm$ %0.2f)' % (mean_auc, std_auc),
            lw=2,
            alpha=.8)

    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Courbe ROC des différents classifieurs')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
# Dictionnaire de classifieurs pour comparer avec
classifieurs = { 'Dummy Classifier': DummyClassifier()}

In [ ]:
# Permet de comparer plusieurs classieurs
plot_roc_curve(classifieurs, X_train_sample, y_train_sample, n_splits=5)